In [1]:
!wget "https://media.githubusercontent.com/media/pradeep-016/ML_Course/refs/heads/main/7)%20Recommendation%20Systems/Collaborative%20Filtering/mobile_recommendation_system_dataset.csv"

--2024-12-06 14:40:48--  https://media.githubusercontent.com/media/pradeep-016/ML_Course/refs/heads/main/7)%20Recommendation%20Systems/Collaborative%20Filtering/mobile_recommendation_system_dataset.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379386 (1.3M) [text/plain]
Saving to: ‘mobile_recommendation_system_dataset.csv’

mobile_recommendati 100%[===================>]   1.32M  4.77MB/s    in 0.3s    

2024-12-06 14:40:49 (4.77 MB/s) - ‘mobile_recommendation_system_dataset.csv’ saved [1379386/1379386]



In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357261 sha256=560f51597fb072af3e0516eb9dc732ad4f15322ec05bf0e53b66dd7e1be651c1
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [4]:
data = pd.read_csv('mobile_recommendation_system_dataset.csv')

In [5]:
data.head()

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


In [6]:
num_users = 10
data['user_id'] = np.random.randint(1, num_users + 1, len(data))

In [7]:
data['item_id'] = data['name'].factorize()[0]+1

In [8]:
data['price'] = data['price'].replace(r'[^\d]','',regex=True).astype(float)

In [9]:
data.isna().sum()

,0
name,0
ratings,0
price,0
imgURL,0
corpus,12
user_id,0
item_id,0


In [10]:
data.dropna(inplace=True)

In [11]:
reader = Reader(rating_scale = (1, 5))
surprise_data = Dataset.load_from_df(data[['user_id','item_id', 'ratings']],reader)

In [12]:
train_set, test_set = train_test_split(surprise_data, test_size = 0.2, random_state = 42)

In [13]:
model = SVD()
model.fit(train_set)

In [14]:
pred = model.test(test_set)

In [15]:
rmse = accuracy.rmse(pred)
print('Root Mean Squared Error:',rmse)

RMSE: 0.1945
Root Mean Squared Error: 0.19454014977001388


In [16]:
def recommend(user_id, n=5):
  all_items = data['item_id'].unique()
  rated_items = data[data['user_id'] == user_id]['item_id']
  unrated_items = [item for item in all_items if item not in rated_items]

  predictions = [(item, model.predict(user_id, item).est) for item in unrated_items]

  top_n = sorted(predictions, key = lambda x:x[1],reverse = True)[:n]
  return top_n

In [17]:
user_id = 1
top_recommendations = recommend(user_id)

In [18]:
print(f"\nTop-5 Recommendations for User {user_id}:")
for item, rating in top_recommendations:
    print(f"Item {item} with predicted rating {rating:.2f}")


Top-5 Recommendations for User 1:
Item 1937 with predicted rating 4.50
Item 1571 with predicted rating 4.49
Item 329 with predicted rating 4.48
Item 1930 with predicted rating 4.48
Item 1063 with predicted rating 4.48
